In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, countDistinct, sum, avg, row_number, to_date, lit, concat, expr
from pyspark.sql.window import Window
from pyspark.sql.types import TimestampType, IntegerType, DoubleType

In [3]:
spark = SparkSession.builder \
    .appName("Music Streaming KPI Calculation") \
    .getOrCreate()

25/03/20 16:55:07 WARN Utils: Your hostname, brempong-HP-EliteBook-840-G7-Notebook-PC resolves to a loopback address: 127.0.1.1; using 192.168.36.43 instead (on interface wlp0s20f3)
25/03/20 16:55:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/20 16:55:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
users = spark.read.csv("data/users/users.csv", header=True, inferSchema=True)
songs = spark.read.csv("data/songs/songs.csv", header=True, inferSchema=True)
streams1 = spark.read.csv("data/streams/streams1.csv", header=True, inferSchema=True)
streams2 = spark.read.csv("data/streams/streams2.csv", header=True, inferSchema=True)
streams3 = spark.read.csv("data/streams/streams3.csv", header=True, inferSchema=True)

In [5]:
streams = streams1.union(streams2).union(streams3)

In [6]:
streaming_data = streams.join(users, streams.user_id == users.user_id, "inner") \
    .join(songs, streams.track_id == songs.track_id, "inner") 

In [7]:
streaming_data.show()

25/03/20 16:55:17 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------------------+-------------------+-------+------------------+--------+-------------+----------+-----+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+-----------+
|user_id|            track_id|        listen_time|user_id|         user_name|user_age| user_country|created_at|   id|            track_id|             artists|          album_name|          track_name|popularity|duration_ms|explicit|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|track_genre|
+-------+--------------------+-------------------+-------+------------------+--------+-------------+----------+-----+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+------+---+--------+--

In [8]:
streaming_data = streaming_data.withColumn("processing_date", to_date(col("created_at")))

In [9]:
streaming_data = streaming_data.withColumn("duration_seconds", col("duration_ms") / 1000)

In [10]:
users.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_age: integer (nullable = true)
 |-- user_country: string (nullable = true)
 |-- created_at: date (nullable = true)



In [11]:
songs.printSchema()

root
 |-- id: integer (nullable = true)
 |-- track_id: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- album_name: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- duration_ms: string (nullable = true)
 |-- explicit: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- key: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: string (nullable = true)
 |-- valence: string (nullable = true)
 |-- tempo: double (nullable = true)
 |-- time_signature: double (nullable = true)
 |-- track_genre: string (nullable = true)



In [12]:
streams.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- track_id: string (nullable = true)
 |-- listen_time: timestamp (nullable = true)



In [15]:
import pandas as pd
df = pd.read_parquet("s3://music-streaming-data-dynamo/processed_data/streams1/listen_date=2024-06-25/part-00000-a284e2e6-27c2-4c61-942f-5b0c645858e3.c000.snappy.parquet", engine='pyarrow')   

In [16]:
df.head()

,track_id,user_id,listen_time,partition_0,user_name,user_age,user_country,created_at,id,artists,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,processing_date,duration_seconds,listen_date
0,4vAnqYqm1DOo5IDVcxORX3,25675,2024-06-25 00:02:08,streams1,Megan Perez,33,United States,2024-07-22,206,Canyon City,...,0.878,0.000473,0.1120,0.224,136.110,4,acoustic,2025-03-20,144.830,2024-06-25
1,5himtcBG5IetEOAYuB1Lu1,35526,2024-06-25 19:33:07,streams1,Kelly Burns,38,United States,2024-01-17,703,SafetySuit,...,0.872,0.0,0.0918,0.510,133.870,4,acoustic,2025-03-20,175.200,2024-06-25
2,3tBS9yN7auzP4PeAEzfGEg,14461,2024-06-25 02:18:04,streams1,Michael Allen,20,United States,2024-03-17,727,Takehara Pistol,...,0.0299,0.0,0.2300,0.617,88.002,4,acoustic,2025-03-20,182.306,2024-06-25
3,5K0z3idLxEFY0tv82NeMPe,41366,2024-06-25 03:44:34,streams1,Elizabeth Chen,36,United States,2024-03-30,1080,Criolo,...,0.525,8.57e-06,0.7110,0.681,94.006,4,afrobeat,2025-03-20,267.480,2024-06-25
4,4dKAXHyIN8xkLTl5OIL9FN,49723,2024-06-25 01:36:49,streams1,Jonathan Williams,52,United States,2024-12-18,1893,Mulatu Astatke,...,0.871,0.878,0.0596,0.741,86.564,4,afrobeat,2025-03-20,333.400,2024-06-25


In [18]:
df.columns

Index(['track_id', 'user_id', 'listen_time', 'partition_0', 'user_name',
       'user_age', 'user_country', 'created_at', 'id', 'artists', 'album_name',
       'track_name', 'popularity', 'duration_ms', 'explicit', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'track_genre', 'processing_date', 'duration_seconds', 'listen_date'],
      dtype='object')